# DSSM
# data process

In [2]:
import datetime
import numpy as np
import pandas as pd
import joblib
import warnings
import logging
import os
import gc
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import collections
import re
import copy
import torch
import random
import itertools

import utils_

from functools import reduce
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from joblib import Parallel, delayed
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.inspection import permutation_importance


# pd.set_option('display.max_columns', None)
# pd.set_option('max_row', 500)
warnings.filterwarnings('ignore')
tqdm.pandas(desc='pandas bar')

In [ ]:
torch.__version__

# 数据处理

* 样本id、绑定物料，正样本

In [ ]:
# df = pd.read_csv('../data/sample_bind_card_20221120_20221211.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_20221218.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_20230101.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_20230219.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_20230226.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_20230604_20230625.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_20230702.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_increment_20230702.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_20230730_20230820.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_20230827.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_increment_20230910.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/sample_bind_card_increment_20230813_20230903.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/other/balance_26/sample_bind_card_20231008_20231029.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/other/balance_26/sample_bind_card_20231105.txt', sep='\t', encoding='utf-8')
# df = pd.read_csv('../data/other/balance_33/sample_bind_card_20240218_20240310.txt', sep='\t', encoding='utf-8')
df = pd.read_csv('../data/other/balance_33/sample_bind_card_20240225_20240317.txt', sep='\t', encoding='utf-8')

# df_1 = pd.read_csv('../data/sample_bind_card_20220925_20221113.txt', sep='\t', encoding='utf-8')
# df_2 = pd.read_csv('../data/sample_bind_card_20221120_20221211.txt', sep='\t', encoding='utf-8')
# print(df_1.shape)
# print(df_2.shape)

# df = pd.concat([df_1, df_2], axis=0)
# df.reset_index(drop=True, inplace=True)

print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
df[['obs_dt', 'dt', 'uid']].groupby(by=['obs_dt', 'dt']).count()

In [ ]:
df['card'].unique()

In [ ]:
df['action_type'].value_counts()

In [ ]:
df['role'].value_counts()

In [ ]:
# 目标物料（新）
list_card = [
    'xxx'
]
print(len(list_card))

In [ ]:
df_target = df[
    (df['card'].isin(list_card))
    # &(df['role']==1)
    # &(~df['action_type'].isin([24, 29, 31]))
]
df_target.reset_index(drop=True, inplace=True)
print(df_target.shape)
df_target.head()

In [ ]:
df_target['action_type'].value_counts()

In [ ]:
df_target['role'].value_counts()

In [ ]:
df_target['card'].value_counts()

In [ ]:
df_target[df_target['card']=='xxx'][['obs_dt', 'dt', 'uid']].groupby(by=['obs_dt', 'dt']).count()

* 非目标物料手动生成负样本

In [ ]:
ser_card = df_target['card'].value_counts()
dict_card_num = {k:v for k, v in ser_card.items()}
dict_card_num

In [ ]:
list_card = list(dict_card_num.keys())
list_card

In [ ]:
dict_card_weight = {x:round(dict_card_num[x]/min(dict_card_num.values())) for x in list_card}
dict_card_weight

In [ ]:
def func_combine(df_group, neg_ratio, replacement):
    uid = df_group.head(1)['uid'].max()
    obs_dt = df_group.head(1)['obs_dt'].max()
    
    list_target_pos = list(set(df_group['card']))
    list_neg = list(itertools.chain.from_iterable([[x]*dict_card_weight[x] for x in list_card if x not in list_target_pos]))
    
    if replacement:
        coef = len(list_target_pos) * neg_ratio
        list_target_neg = []
        
        for i in range(coef):
            list_target_neg.append(random.sample(list_neg, 1)[0])
    else:
        coef = len(list_target_pos) * neg_ratio // len(list_neg) + 1
        list_neg = list_neg * coef

        list_target_neg = random.sample(list_neg, len(list_target_pos)*neg_ratio)
    
    df_pos = pd.DataFrame({'uid': [uid] * len(list_target_pos), 
                           'obs_dt': [obs_dt] * len(list_target_pos), 
                           'card': list_target_pos, 
                           'label': [1] * len(list_target_pos)})
    df_neg = pd.DataFrame({'uid': [uid] * len(list_target_neg), 
                           'obs_dt': [obs_dt] * len(list_target_neg), 
                           'card': list_target_neg, 
                           'label': [0] * len(list_target_neg)})
    
    df_res = pd.concat([df_pos, df_neg], axis=0)
    
    return df_res


def func_combine_pos(df_group, neg_ratio, replacement):
    uid = df_group.head(1)['uid'].max()
    obs_dt = df_group.head(1)['obs_dt'].max()
    
    list_target_pos = list(set(df_group['card']))
    
    df_pos = pd.DataFrame({'uid': [uid] * len(list_target_pos), 
                           'obs_dt': [obs_dt] * len(list_target_pos), 
                           'card': list_target_pos, 
                           'label': [1] * len(list_target_pos)})
    
    return df_pos


def apply_parallel(df_grouped, func, neg_ratio=1, replacement=False):
    try:
        with tqdm(df_grouped) as t:
            list_paraller = Parallel(n_jobs=8)(delayed(func)(df_group, neg_ratio, replacement) for name, df_group in t)
    except KeyboardInterrupt:
        t.close()
        raise
    t.close()
    
    return list_paraller

In [ ]:
# 按周合并后去重
df_target_grouped = df_target.groupby(by=['uid', 'obs_dt'])
list_df_target_grouped_combine = apply_parallel(df_target_grouped, func_combine)
# list_df_target_grouped_combine = apply_parallel(df_target_grouped, func_combine, neg_ratio=3)
# list_df_target_grouped_combine = apply_parallel(df_target_grouped, func_combine, neg_ratio=3, replacement=True)
# list_df_target_grouped_combine = apply_parallel(df_target_grouped, func_combine, neg_ratio=5)
# list_df_target_grouped_combine = apply_parallel(df_target_grouped, func_combine_pos)
df_target_grouped_combine = pd.concat(list_df_target_grouped_combine, axis=0)
df_target_grouped_combine.reset_index(drop=True, inplace=True)
print(df_target_grouped_combine.shape)
df_target_grouped_combine.head()

In [ ]:
pd.set_option('max_row', 500)
df_target_grouped_combine[['card', 'label', 'uid']].groupby(by=['card', 'label']).count()

In [ ]:
df_target_grouped_combine['label'].value_counts()

In [ ]:
# 814989 / 271663
# 124629 / 41543
# 1358315 / 271663
# 329055 / 329055
# 51364 / 51364
# 63678 / 63678
# 148555 / 148555
# 575690 / 575690
# 870452 / 870452
# 710879 / 710879
# 569717 / 569717
# 589657 / 589657
842647 / 842647

In [ ]:
# 正样本处理
df_target_grouped_combine_pos = df_target_grouped_combine[df_target_grouped_combine['label']==1]
df_target_grouped_combine_pos.reset_index(drop=True, inplace=True)
print(df_target_grouped_combine_pos.shape)
df_target_grouped_combine_pos.head()

In [ ]:
# 正样本处理
# 正样本周去重
df_pos = df_target.groupby(by=['uid', 'obs_dt', 'card']).count()
df_pos.reset_index(inplace=True)
df_pos = df_pos[['uid', 'obs_dt', 'card']]
df_pos['label'] = 1
print(df_pos.shape)
df_pos.head()

In [ ]:
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_20221120_20221211.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_20221218.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_20221120_20221211.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_20221218.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_3_neg_20221120_20221211.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_3_neg_20221218.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_5_neg_20221120_20221211.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_22_20221120_20221211.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_24_20230604_20230625.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_25_20230730_20230820.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_balance_24_20230730_20230820.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/df_sample_label_increment_balance_24_20230813_20230903.pickle')
# utils_.save_pickle(df_target_grouped_combine, '../data/other/balance_26/df_sample_label_balance_26_20231008_20231029.pickle')
utils_.save_pickle(df_target_grouped_combine, '../data/other/balance_34/df_sample_label_balance_34_20240225_20240317.pickle')

# utils_.save_pickle(df_target_grouped_combine_pos, '../data/df_sample_label_balance_pos_22_20221218.pickle')
# utils_.save_pickle(df_target_grouped_combine_pos, '../data/df_sample_label_balance_pos_22_20230101.pickle')
# utils_.save_pickle(df_target_grouped_combine_pos, '../data/df_sample_label_balance_pos_22_20230219.pickle')
# utils_.save_pickle(df_pos, '../data/df_sample_label_pos_22_20220925_20221211.pickle')
# utils_.save_pickle(df_pos, '../data/df_sample_label_pos_22_20230226.pickle')
# utils_.save_pickle(df_pos, '../data/df_sample_label_pos_24_20230702.pickle')
# utils_.save_pickle(df_pos, '../data/df_sample_label_pos_24_20230702.pickle')
# utils_.save_pickle(df_pos, '../data/df_sample_label_increment_pos_24_20230702.pickle')
# utils_.save_pickle(df_pos, '../data/df_sample_label_pos_25_20230827.pickle')
# utils_.save_pickle(df_pos, '../data/df_sample_label_increment_pos_24_20230910.pickle')
# utils_.save_pickle(df_pos, '../data/other/balance_26/df_sample_label_pos_26_20231105.pickle')

In [ ]:
# df_target_grouped_combine = utils_.load_pickle('../data/df_sample_label_balance_24_20230604_20230625.pickle')
# df_target_grouped_combine = utils_.load_pickle('../data/df_sample_label_balance_25_20230730_20230820.pickle')
# df_target_grouped_combine = utils_.load_pickle('../data/df_sample_label_balance_24_20230730_20230820.pickle')
# df_target_grouped_combine = utils_.load_pickle('../data/df_sample_label_increment_balance_24_20230813_20230903.pickle')
# df_target_grouped_combine = utils_.load_pickle('../data/other/balance_26/df_sample_label_balance_26_20231008_20231029.pickle')
df_target_grouped_combine = utils_.load_pickle('../data/other/balance_34/df_sample_label_balance_34_20240225_20240317.pickle')
print(df_target_grouped_combine.shape)
df_target_grouped_combine.head()

# df_pos = utils_.load_pickle('../data/df_sample_label_pos_24_20230702.pickle')
# df_pos = utils_.load_pickle('../data/df_sample_label_increment_pos_24_20230702.pickle')
# df_pos = utils_.load_pickle('../data/df_sample_label_pos_25_20230827.pickle')
# df_pos = utils_.load_pickle('../data/df_sample_label_increment_pos_24_20230910.pickle')
# df_pos = utils_.load_pickle('../data/other/balance_26/df_sample_label_pos_26_20231105.pickle')
# print(df_pos.shape)
# df_pos.head()

In [ ]:
aaa = df_pos[['uid']].head(10)
aaa

In [ ]:
aaa.to_csv('uid.txt', sep='\t', encoding='utf-8', index=False, header=False)

In [ ]:
df_target_grouped_combine['dt'] = None
df_target_grouped_combine.head()

# df_pos['dt'] = None
# df_pos.head()

In [ ]:
# df_target_grouped_combine.to_csv('../data/df_sample_label_20221120_20221211.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_20221218.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_balance_20221120_20221211.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_balance_20221218.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_balance_3_neg_20221120_20221211.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_balance_3_neg_20221218.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_balance_22_20221120_20221211.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_balance_24_20230604_20230625.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_balance_25_20230730_20230820.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_balance_24_20230730_20230820.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/df_sample_label_increment_balance_24_20230813_20230903.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine.to_csv('../data/other/balance_26/df_sample_label_balance_26_20231008_20231029.txt', sep='\t', encoding='utf-8', index=False, header=False)
df_target_grouped_combine.to_csv('../data/other/balance_34/df_sample_label_balance_34_20240225_20240317.txt', sep='\t', encoding='utf-8', index=False, header=False)

# df_target_grouped_combine_pos.to_csv('../data/df_sample_label_balance_pos_22_20221218.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine_pos.to_csv('../data/df_sample_label_balance_pos_22_20230101.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_target_grouped_combine_pos.to_csv('../data/df_sample_label_balance_pos_22_20230219.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_pos.to_csv('../data/df_sample_label_pos_22_20220925_20221211.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_pos.to_csv('../data/df_sample_label_pos_22_20230226.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_pos.to_csv('../data/df_sample_label_pos_24_20230702.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_pos.to_csv('../data/df_sample_label_increment_pos_24_20230702.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_pos.to_csv('../data/df_sample_label_pos_25_20230827.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_pos.to_csv('../data/df_sample_label_increment_pos_24_20230910.txt', sep='\t', encoding='utf-8', index=False, header=False)
# df_pos.to_csv('../data/other/balance_26/df_sample_label_pos_26_20231105.txt', sep='\t', encoding='utf-8', index=False, header=False)

* 匹配user特征

In [ ]:
df_feats = pd.read_csv('../data/sample_label_feature_bfx_std_balance_20221120_20221211.txt', sep='\t', encoding='utf-8')
print(df_feats.shape)
df_feats.head()

In [ ]:
df_feats = df_feats[df_feats['label']==1][[x for x in df_feats.columns if x not in ['card', 'label']]]
df_feats.reset_index(drop=True, inplace=True)
print(df_feats.shape)
df_feats.head()

In [ ]:
df_feats_unique = df_feats.groupby(by=['uid', 'obs_dt']).max()
df_feats_unique.reset_index(inplace=True)
print(df_feats_unique.shape)
df_feats_unique.head()

In [ ]:
utils_.save_pickle(df_feats_unique, '../data/df_feats_20221120_20221211.pickle')

* item特征

In [ ]:
# df_item = pd.read_excel('../data/other/balance/dssm_item_features/物料特征.xlsx')
df_item = pd.read_excel('../data/other/balance/dssm_item_features_batch_neg_22/物料特征22.xlsx')
print(df_item.shape)
df_item.head()

In [ ]:
df_item['card'] = df_item['物料']
df_item['card_id'] = df_item['物料编码']
df_item.head()

In [ ]:
# 物料类型，card_type
df_item['card_type'] = df_item['物料类型']
df_item['card_type'].value_counts()

In [ ]:
dict_feats_card_type = {
    'aaa': 1, 
    'bbb': 2
}

df_item['card_type'].replace(dict_feats_card_type, inplace=True)
df_item.head()

In [ ]:
df_item[['物料类型', 'card_type', 'card']].groupby(['物料类型', 'card_type']).count()

In [ ]:
# 物料属性，item_type
df_item['item_type'] = df_item['物料属性']
df_item['item_type'].value_counts()

In [ ]:
dict_feats_item_type = {
    '国有': 1, 
    '股份制': 2, 
    '城商': 3
}

df_item['item_type'].replace(dict_feats_item_type, inplace=True)
df_item.head()

In [ ]:
df_item[['物料属性', 'item_type', 'card']].groupby(['物料属性', 'item_type']).count()

In [ ]:
# 物料成立时间, item_create_time
df_item['物料成立时间'].value_counts()

In [ ]:
df_item['物料成立时期'] = df_item['物料成立时间'].apply(lambda x: '解放前' if x<=1945 
                                                                      else '解放后~改革开放' if x<=1978 
                                                                      else '改革开放~1989' if x <= 1989 
                                                                      else '1989~1999' if x <= 1999 
                                                                      else '21世纪')
df_item[['物料成立时间', '物料成立时期']]

In [ ]:
df_item['item_create_time'] = df_item['物料成立时期']
df_item['item_create_time'].value_counts()

In [ ]:
dict_feats_item_create_time = {
    '解放前': 1, 
    '解放后~改革开放': 2, 
    '改革开放~1989': 3, 
    '1989~1999': 4, 
    '21世纪': 5
}

df_item['item_create_time'].replace(dict_feats_item_create_time, inplace=True)
df_item.head()

In [ ]:
df_item[['物料成立时期', 'item_create_time', 'card']].groupby(['物料成立时期', 'item_create_time']).count()

In [ ]:
# 物料成立地点，item_create_city
df_item['item_create_city'] = df_item['物料成立地点']
df_item['item_create_city'].value_counts()

In [ ]:
dict_feats_item_create_city = {
    '北京': 1, 
    '上海': 2, 
    '广州': 3, 
    '深圳': 4, 
    '杭州': 5, 
    '南京': 6, 
    '长沙': 7, 
    '福州': 8
}

df_item['item_create_city'].replace(dict_feats_item_create_city, inplace=True)
df_item.head()

In [ ]:
df_item[['物料成立地点', 'item_create_city', 'card']].groupby(['物料成立地点', 'item_create_city']).count()

In [ ]:
# 物料人员规模（2022.06），item_people_size_202206
df_item['物料人员规模（2022.06）'].describe()

In [ ]:
df_bins_index, list_bins = utils_.binning(df_item, '物料人员规模（2022.06）', method='st', parameter={'bins_thresholds': [-np.inf, 20000, 50000, 100000, 200000, 500000]})

In [ ]:
df_bins_index

In [ ]:
list_bins

In [ ]:
df_item['item_people_size_202206'] = df_bins_index['物料人员规模（2022.06）_bins_index'] + 1
df_item['item_people_size_202206'].value_counts()

In [ ]:
# 物料注册资本（万元），item_regist_capital
df_item['物料注册资本（万元）'].describe()

In [ ]:
df_bins_index, list_bins = utils_.binning(df_item, '物料注册资本（万元）', method='st', parameter={'bins_thresholds': [-np.inf, 2e6, 5e6, 1e7, 5e7]})

In [ ]:
df_bins_index

In [ ]:
list_bins

In [ ]:
df_item['item_regist_capital'] = df_bins_index['物料注册资本（万元）_bins_index'] + 1
df_item['item_regist_capital'].value_counts()

In [ ]:
# 物料年营收（亿，2021FY），item_revenue_annual_2021
df_item['物料年营收（亿，2021FY）'].describe()

In [ ]:
df_bins_index, list_bins = utils_.binning(df_item, '物料年营收（亿，2021FY）', method='st', parameter={'bins_thresholds': [-np.inf, 500, 1000, 2000, 5000, 10000]})

In [ ]:
df_bins_index

In [ ]:
list_bins

In [ ]:
df_item['item_revenue_annual_2021'] = df_bins_index['物料年营收（亿，2021FY）_bins_index'] + 1
df_item['item_revenue_annual_2021'].value_counts()

In [ ]:
# 物料年利润（亿，2021FY），item_profit_annual_2021
df_item['物料年利润（亿，2021FY）'].describe()

In [ ]:
df_bins_index, list_bins = utils_.binning(df_item, '物料年利润（亿，2021FY）', method='st', parameter={'bins_thresholds': [-np.inf, 200, 500, 1000, 5000]})

In [ ]:
df_bins_index

In [ ]:
list_bins

In [ ]:
df_item['item_profit_annual_2021'] = df_bins_index['物料年利润（亿，2021FY）_bins_index'] + 1
df_item['item_profit_annual_2021'].value_counts()

In [ ]:
# 物料年利润（亿，2021FY）占比，item_profit_ratio_annual_2021
df_item['物料年利润（亿，2021FY）占比'] = df_item['物料年利润（亿，2021FY）'] / df_item['物料年营收（亿，2021FY）']
df_item[['物料年营收（亿，2021FY）', '物料年利润（亿，2021FY）', '物料年利润（亿，2021FY）占比']]

In [ ]:
df_item['物料年利润（亿，2021FY）占比'].describe()

In [ ]:
df_bins_index, list_bins = utils_.binning(df_item, '物料年利润（亿，2021FY）占比', method='st', parameter={'bins_thresholds': [-np.inf, 0.3, 0.35, 0.4, 0.45, 0.5]})

In [ ]:
df_bins_index

In [ ]:
list_bins

In [ ]:
df_item['item_profit_ratio_annual_2021'] = df_bins_index['物料年利润（亿，2021FY）占比_bins_index'] + 1
df_item['item_profit_ratio_annual_2021'].value_counts()

In [ ]:
# 股东数量（万，2021.12.31），item_stock_people_size
df_item['股东数量（万，2021.12.31）'].describe()

In [ ]:
df_bins_index, list_bins = utils_.binning(df_item, '股东数量（万，2021.12.31）', method='st', parameter={'bins_thresholds': [-np.inf, 20, 50, 100]})

In [ ]:
df_bins_index

In [ ]:
list_bins

In [ ]:
df_item['item_stock_people_size'] = df_bins_index['股东数量（万，2021.12.31）_bins_index'] + 2
df_item['item_stock_people_size'].value_counts()

In [ ]:
# 是否港股上市，item_is_hk_stock
df_item['item_is_hk_stock'] = df_item['是否港股上市']
df_item['item_is_hk_stock'].value_counts()

In [ ]:
dict_feats_item_is_hk_stock = {
    '未上市': 1, 
    '否': 2, 
    '是': 3
}

df_item['item_is_hk_stock'].replace(dict_feats_item_is_hk_stock, inplace=True)
df_item.head()

In [ ]:
df_item[['是否港股上市', 'item_is_hk_stock', 'card']].groupby(['是否港股上市', 'item_is_hk_stock']).count()

In [ ]:
df_item_ = df_item[['card', 'card_id', 
                    'card_type', 'item_type', 'item_create_time', 'item_create_city', 'item_people_size_202206', 
                    'item_regist_capital', 'item_revenue_annual_2021', 'item_profit_annual_2021', 'item_profit_ratio_annual_2021', 'item_stock_people_size', 
                    'item_is_hk_stock']]
print(df_item_.shape)
df_item_.head()

In [ ]:
# df_item_.to_csv('../data/other/balance/dssm_item_features/item_feats.txt', sep='\t', encoding='utf-8', index=False, header=False)
df_item_.to_csv('../data/other/balance/dssm_item_features_batch_neg_22/item_feats.txt', sep='\t', encoding='utf-8', index=False, header=False)

In [ ]:
# utils_.save_pickle(df_item_, '../data/other/balance/dssm_item_features/item_feats.pickle')
utils_.save_pickle(df_item_, '../data/other/balance/dssm_item_features_batch_neg_22/item_feats.pickle')

In [ ]:
# df_item_ = utils_.load_pickle('../data/other/balance/dssm_item_features/item_feats.pickle')
df_item_ = utils_.load_pickle('../data/other/balance/dssm_item_features_batch_neg_22/item_feats.pickle')
print(df_item_.shape)
df_item_.head()

In [ ]:
# dict_lbe_item = joblib.load('../data/other/balance/dssm_item_features_3_neg/dict_lbe_item.pickle')
# item_sparse_feats = utils_.load_pickle('../data/other/balance/dssm_item_features_3_neg/list_item_sparse_feats.pickle')
dict_lbe_item = joblib.load('../data/other/balance/dssm_item_features_batch_neg_22/dict_lbe_item.pickle')
item_sparse_feats = utils_.load_pickle('../data/other/balance/dssm_item_features_batch_neg_22/list_item_sparse_feats.pickle')
for x in item_sparse_feats:
    print(x, dict_lbe_item[x].classes_)

In [ ]:
list_X_all_item_sparse = []

try:
    with tqdm(item_sparse_feats) as t:
        for x in t:
            # list_feat_values_unseen = list(set(df_user_item[x].unique())-set(dict_lbe_item[x].classes_))
            # if len(list_feat_values_unseen) > 0:
            #     print(x)
            #     df_user_item[x].replace(list_feat_values_unseen, 16, inplace=True)
            df_X_all_item_sparse_each = pd.DataFrame(dict_lbe_item[x].transform(df_item_[x]), columns=[x])
            list_X_all_item_sparse.append(df_X_all_item_sparse_each)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_X_all_item_sparse = pd.concat([df_item_[['card']]]+list_X_all_item_sparse, axis=1)
print(df_X_all_item_sparse.shape)
df_X_all_item_sparse

In [ ]:
df_X_all_item_sparse.to_csv('../data/other/balance/dssm_item_features/item_feats_transform.csv', encoding='utf-8', index=False)
df_X_all_item_sparse.to_excel('../data/other/balance/dssm_item_features/item_feats_transform.xlsx', encoding='utf-8', index=False)